# Diseño de Dataframes para los Endpoints

## importacion, limpieza y normalizacion de las tablas a utilizar

* importamos la librerias

In [1]:
import pandas as pd

* hacemos la carga de archivos

In [2]:
df_games = pd.read_parquet("../Datasets_finales/games_steam.parquet")
df_items = pd.read_parquet("../Datasets_finales/user_items.parquet")
df_reviews = pd.read_parquet("../Datasets_endpoints/user_reviews_sentiment.parquet")

In [3]:
df_items.head(2)

,item_id,playtime_forever,user_id,items_count,steam_id
0,10,6.0,76561197970982479,277,76561197970982479
1,20,0.0,76561197970982479,277,76561197970982479


In [4]:
df_reviews.head(2)

,user_id,item_id,recommend,sentiment_analysis
0,76561197970982479,1250,True,2
1,76561197970982479,22200,True,2


In [5]:
df_games.head(2)

,genres,app_name,release_date,price,id_item,developer
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,free,643980,Secret Level SRL


### tablas finales para el uso de las consultas
![Tabla dividido por las solicitudes de las consulta](../_src/Tabla_Endpoints.jpg)

### Limpieza y Normalizacion de Games

* **funcion para tomar el primer elemento de la lista de generos**

In [6]:
def primer_genre(lista):
    if isinstance(lista, object) and lista is not None:
        return lista[0]
    else:
        return None

* sacamos el primer genero de cada juego

In [7]:
df_games["genre"] = df_games['genres'].apply(primer_genre)

* sacamos el año de release_date y los nulos lo ponemos en 0 y convertimos en enteros a los años

In [8]:
df_games["año"] = df_games["release_date"].dt.year
df_games["año"] = df_games["año"].fillna(0)
df_games["año"] = df_games["año"].astype(int)

* vamos a renombrar id_item a item_id y reordenar las columnas

In [9]:
columnas_a_eliminar_games = ["genres","app_name","release_date"]
columnas_a_eliminar_items = ["items_count","steam_id"]
df_games = df_games.drop(columnas_a_eliminar_games,axis=1)
df_items = df_items.drop(columnas_a_eliminar_items,axis=1)

In [10]:
df_games.rename(columns={"id_item":"item_id"},inplace=True)
ordenar_games = ["item_id","price","genre","año","developer"]
df_games = df_games[ordenar_games]
df_games.head(5)

,item_id,price,genre,año,developer
0,761140,4.99,Action,2018,Kotoshiro
1,643980,free,Free to Play,2018,Secret Level SRL
2,670290,free,Casual,2017,Poolians.com
3,767400,0.99,Action,2017,彼岸领域
4,773570,2.99,None,0,None


* convertimos a minusculas la columna de developer y genre por si se pone mal alguna letra en formato mayuscula-minuscula

In [11]:
df_games["developer"] = df_games["developer"].str.lower()
df_games["genre"] = df_games["genre"].str.lower()

In [12]:
df_games.head(5)

,item_id,price,genre,año,developer
0,761140,4.99,action,2018,kotoshiro
1,643980,free,free to play,2018,secret level srl
2,670290,free,casual,2017,poolians.com
3,767400,0.99,action,2017,彼岸领域
4,773570,2.99,None,0,None


### Limpieza y Normalizacion de Items

* hacemos un chequeo de los nulos de la tabla items y borramos los datos nulos que seas necesarios

In [13]:
df_items[df_items["item_id"].isna()].head(5)

,item_id,playtime_forever,user_id
3733,None,NaN,Wackky
3849,None,NaN,76561198079601835
6019,None,NaN,hellom8o
6523,None,NaN,starkillershadow553
7237,None,NaN,darkenkane


In [14]:
df_items[df_items["playtime_forever"].isna()].head(5)

,item_id,playtime_forever,user_id
3733,None,NaN,Wackky
3849,None,NaN,76561198079601835
6019,None,NaN,hellom8o
6523,None,NaN,starkillershadow553
7237,None,NaN,darkenkane


* tanto item_id como playtime_forever tienen nulos, lo mas importante es limpiar los item_id ya que es importante al igual que user_id.

In [15]:
df_items.dropna(subset="item_id",inplace=True)

* hacemos un chequeo de los tipo de formato que estan los datos, y vamos a convertir el item_id en enteros

In [16]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5094082 entries, 0 to 5110794
Data columns (total 3 columns):
 #   Column            Dtype  
---  ------            -----  
 0   item_id           object 
 1   playtime_forever  float64
 2   user_id           object 
dtypes: float64(1), object(2)
memory usage: 155.5+ MB


In [17]:
df_items["item_id"] = df_items["item_id"].astype(int)

* hacemos una normalizacion a minuscula de los User

In [18]:
df_items["user_id"] = df_items["user_id"].str.lower()

* ya quedó listo nuestra tabla items

In [19]:
df_items.head(5)

,item_id,playtime_forever,user_id
0,10,6.0,76561197970982479
1,20,0.0,76561197970982479
2,30,7.0,76561197970982479
3,40,0.0,76561197970982479
4,50,0.0,76561197970982479


### Limpieza y normalizacion de Reviews

* Hacemos un chequeo de los nulos y hacemos la limpieza

In [20]:
df_reviews[df_reviews["item_id"].isna()].head(5)

,user_id,item_id,recommend,sentiment_analysis
137,gdxsd,None,None,1
177,76561198094224872,None,None,1
2558,76561198021575394,None,None,1
9956,cmuir37,None,None,1
13528,Jaysteeny,None,None,1


In [21]:
df_reviews.dropna(subset=["item_id"],inplace=True)
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58430 entries, 0 to 58457
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             58430 non-null  object
 1   item_id             58430 non-null  object
 2   recommend           58430 non-null  object
 3   sentiment_analysis  58430 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 2.2+ MB


* convertimos user_id a minusculas

In [22]:
df_reviews["user_id"] = df_reviews["user_id"].str.lower()

* ya tenemos la tabla limpio de nulos, a lo que procedemos a convertir los item_id en formato entero 

In [23]:
df_reviews["item_id"] = df_reviews["item_id"].astype(int)
df_reviews.head()

,user_id,item_id,recommend,sentiment_analysis
0,76561197970982479,1250,True,2
1,76561197970982479,22200,True,2
2,76561197970982479,43110,True,2
3,js41637,251610,True,2
4,js41637,227300,True,2


### **ya tenemos todas las tablas limpias hasta donde se puede para no evitar perdida de datos y la normalización para la futura unión de las tablas**

## Diseño de Endpoints para las consultas de la API

### Consulta 1: Def developer(desarrollador: str), Debe devolver Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora.

* copiamos la tabla de games y creamos el endpoint para la consulta 1

In [24]:
df_developer = df_games.copy()
df_developer

,item_id,price,genre,año,developer
0,761140,4.99,action,2018,kotoshiro
1,643980,free,free to play,2018,secret level srl
2,670290,free,casual,2017,poolians.com
3,767400,0.99,action,2017,彼岸领域
4,773570,2.99,None,0,None
...,...,...,...,...,...
32128,773640,1.99,casual,2018,"nikita ""ghost_rus"""
32129,733530,4.99,casual,2018,sacada
32130,610660,1.99,indie,2018,laush dmitriy sergeevich
32131,658870,4.99,casual,2017,"xropi,stev3ns"


* no vamos a necesitar la columna genero ya que no lo vamos a utilizar

In [25]:
df_developer.drop("genre",axis=1,inplace=True)
df_developer.head(5)

,item_id,price,año,developer
0,761140,4.99,2018,kotoshiro
1,643980,free,2018,secret level srl
2,670290,free,2017,poolians.com
3,767400,0.99,2017,彼岸领域
4,773570,2.99,0,None


* hacemos un chequeo de nulos en las columnas developer y año

In [26]:
df_developer[df_developer["developer"].isna()].head(5)

,item_id,price,año,developer
4,773570,2.99,0,None
11,724910,free,0,None
19,772590,4.99,0,None
20,640250,2.99,0,None
22,711440,0.99,0,None


In [27]:
df_developer[df_developer["año"] == 0]

,item_id,price,año,developer
4,773570,2.99,0,None
10,768570,None,0,qucheza
11,724910,free,0,None
19,772590,4.99,0,None
20,640250,2.99,0,None
...,...,...,...,...
32085,755830,None,0,"greyson richey,nicholas lives"
32086,708070,None,0,rechargecomplete
32095,250440,None,0,the amiable
32121,772180,None,0,versovr


* Limpiamos los valores nulos de developer y los años que tienen 0

In [28]:
df_developer.dropna(subset=["developer"],inplace=True)
df_developer = df_developer[df_developer["año"] != 0]
df_developer.reset_index(drop=True)

,item_id,price,año,developer
0,761140,4.99,2018,kotoshiro
1,643980,free,2018,secret level srl
2,670290,free,2017,poolians.com
3,767400,0.99,2017,彼岸领域
4,772540,3.99,2018,trickjump games ltd
...,...,...,...,...
28528,745400,1.99,2018,bidoniera games
28529,773640,1.99,2018,"nikita ""ghost_rus"""
28530,733530,4.99,2018,sacada
28531,610660,1.99,2018,laush dmitriy sergeevich


In [29]:
df_developer.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28533 entries, 0 to 32131
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   item_id    28533 non-null  int32 
 1   price      27558 non-null  object
 2   año        28533 non-null  int32 
 3   developer  28533 non-null  object
dtypes: int32(2), object(2)
memory usage: 891.7+ KB


* **Ya terminamos de hacer la tabla que se va a usar en la consulta 1**

### Consulta 2: def userdata (user_id: str), Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

In [30]:
df_userdata_review = df_reviews.copy() 
df_userdata_items = df_items.copy()
df_userdata_games = df_games.copy()

In [31]:
df_userdata_games.head(2)

,item_id,price,genre,año,developer
0,761140,4.99,action,2018,kotoshiro
1,643980,free,free to play,2018,secret level srl


In [32]:
df_userdata_items.head(2)

,item_id,playtime_forever,user_id
0,10,6.0,76561197970982479
1,20,0.0,76561197970982479


In [33]:
df_userdata_review.head(2)

,user_id,item_id,recommend,sentiment_analysis
0,76561197970982479,1250,True,2
1,76561197970982479,22200,True,2


* **columnas a utilizar de cada tabla:**
- games : price, item_id
- items : item_id y user_id
- review : item_id, ,user_id,recommend

* **la tabla final debe quedar:**
- user_id,item_id,price,recommend

* procedemos a hacer la limpieza de columnas que no vayamos a utilizar

In [34]:
a_eliminar_games =["genre","año","developer"]
df_userdata_games.drop(a_eliminar_games,axis=1,inplace=True)
df_userdata_items.drop("playtime_forever",axis=1,inplace=True)
df_userdata_review.drop("sentiment_analysis",axis=1,inplace=True)

* hacemos un chequeo de nulos y normalizacion de datos para evitar confictos a la de unir las tablas

In [35]:
df_userdata_review.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58430 entries, 0 to 58457
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    58430 non-null  object
 1   item_id    58430 non-null  int32 
 2   recommend  58430 non-null  object
dtypes: int32(1), object(2)
memory usage: 1.6+ MB


* primero vamos a hacer la union de la tabla por coincidentes del id del item

In [36]:
df_userdata_union = pd.merge(df_userdata_games,df_userdata_items,on="item_id") 
df_userdata_union.head(10)

,item_id,price,user_id
0,282010,9.99,utnerd24
1,282010,9.99,i_did_911_just_saying
2,282010,9.99,76561197962104795
3,282010,9.99,r3ap3r78
4,282010,9.99,saint556
5,282010,9.99,chidvd
6,282010,9.99,aerpub
7,282010,9.99,lucifer666678
8,282010,9.99,76561198014848587
9,282010,9.99,filthydeath


* chequeamos si quedó bien la union de la tabla de uno a muchos y se ve que quedó perfecto

In [37]:
df_userdata_union[df_userdata_union["user_id"] == "UTNerd24"].head(5)

,item_id,price,user_id


In [38]:
df_userdata_union[df_userdata_union["item_id"] == 282010].head(5)

,item_id,price,user_id
0,282010,9.99,utnerd24
1,282010,9.99,i_did_911_just_saying
2,282010,9.99,76561197962104795
3,282010,9.99,r3ap3r78
4,282010,9.99,saint556


* al hacer un analisis visual vemos que tanto item_id como user_id son relaciones muchos a muchos. ahora vamos a hacer la ultima union de la tabla con reviews

In [39]:
df_userdata_final = pd.merge(df_userdata_union,df_userdata_review,on=["user_id","item_id"]).sort_values(by="item_id")

In [40]:
df_userdata_final.reset_index(drop=True,inplace=True)
df_userdata_final.head(5)

,item_id,price,user_id,recommend
0,10,9.99,76561198107235245,True
1,10,9.99,76561198085715384,True
2,10,9.99,fuckingdinasaww,True
3,10,9.99,mixadance,True
4,10,9.99,therealchris02,True


* **Ya terminamos la tabla de user_data donde finalmente se pudo hacer la union de las 3 tablas** 

### Consulta 3: def userforgenre (genre: str), Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

In [41]:
df_userforgenre_games = df_games.copy()
df_userforgenre_items = df_items.copy()

In [42]:
df_userforgenre_games.head()

,item_id,price,genre,año,developer
0,761140,4.99,action,2018,kotoshiro
1,643980,free,free to play,2018,secret level srl
2,670290,free,casual,2017,poolians.com
3,767400,0.99,action,2017,彼岸领域
4,773570,2.99,None,0,None


In [43]:
df_userforgenre_items.head()

,item_id,playtime_forever,user_id
0,10,6.0,76561197970982479
1,20,0.0,76561197970982479
2,30,7.0,76561197970982479
3,40,0.0,76561197970982479
4,50,0.0,76561197970982479


* borramos las columnas que no vayamos a utilizar y chequeamos como quedó

In [44]:
df_userforgenre_games.drop(["developer","price"],axis=1,inplace=True)
df_userforgenre_games

,item_id,genre,año
0,761140,action,2018
1,643980,free to play,2018
2,670290,casual,2017
3,767400,action,2017
4,773570,None,0
...,...,...,...
32128,773640,casual,2018
32129,733530,casual,2018
32130,610660,indie,2018
32131,658870,casual,2017


* hacemos la union de las tablas y ordenamos por item_id

In [45]:
df_userforgenre_final = pd.merge(df_userforgenre_games,df_userforgenre_items,on='item_id',sort=True)
df_userforgenre_final.head(5)

,item_id,genre,año,playtime_forever,user_id
0,10,action,2000,6.0,76561197970982479
1,10,action,2000,0.0,js41637
2,10,action,2000,0.0,riot-punch
3,10,action,2000,93.0,doctr
4,10,action,2000,108.0,corrupted_soul


* convertimos los valores nulos de genre en "desconocido" y posteriormente borramos los que tengan con genre desconocido
* eliminamos las filas que tienen año 0

In [46]:
df_userforgenre_final["genre"] = df_userforgenre_final["genre"].fillna("desconocido")
df_userforgenre_final= df_userforgenre_final[(df_userforgenre_final["genre"] != "desconocido")]
df_userforgenre_final = df_userforgenre_final[(df_userforgenre_final["año"] != 0)]

* reseteamos el index para tener un mejor orden en el index

In [47]:
df_userforgenre_final.reset_index(drop=True)
df_userforgenre_final

,item_id,genre,año,playtime_forever,user_id
0,10,action,2000,6.0,76561197970982479
1,10,action,2000,0.0,js41637
2,10,action,2000,0.0,riot-punch
3,10,action,2000,93.0,doctr
4,10,action,2000,108.0,corrupted_soul
...,...,...,...,...,...
4253367,527900,adventure,2016,44.0,76561198159037032
4253374,528660,action,2016,182.0,sirexpower
4253375,528660,action,2016,0.0,thugnificent
4253377,530720,casual,2016,0.0,csmisbeast


* borramos la columna item_id porque ya no nos hace falta

In [48]:
df_userforgenre_final.drop("item_id",axis=1,inplace=True)

* hacemos un chequeo de como quedó la tabla y quedó todo perfecto

In [49]:
df_userforgenre_final.head(5)

,genre,año,playtime_forever,user_id
0,action,2000,6.0,76561197970982479
1,action,2000,0.0,js41637
2,action,2000,0.0,riot-punch
3,action,2000,93.0,doctr
4,action,2000,108.0,corrupted_soul


### Consulta 4: def best_developer_year (año : int), Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)

In [50]:
df_best_developer_year_review = df_reviews.copy()
df_best_developer_year_games = df_games.copy()

In [51]:
df_best_developer_year_review.head(2)

,user_id,item_id,recommend,sentiment_analysis
0,76561197970982479,1250,True,2
1,76561197970982479,22200,True,2


In [52]:
df_best_developer_year_games.head(2)

,item_id,price,genre,año,developer
0,761140,4.99,action,2018,kotoshiro
1,643980,free,free to play,2018,secret level srl


In [53]:
df_best_developer_year_games.drop(["genre","price"],axis=1,inplace=True)
df_best_developer_year_games.head(5)

,item_id,año,developer
0,761140,2018,kotoshiro
1,643980,2018,secret level srl
2,670290,2017,poolians.com
3,767400,2017,彼岸领域
4,773570,0,None


In [54]:
df_best_developer_year_games = df_best_developer_year_games[(df_best_developer_year_games["año"] != 0) & (df_best_developer_year_games["developer"].notna())]
df_best_developer_year_games.head(5)

,item_id,año,developer
0,761140,2018,kotoshiro
1,643980,2018,secret level srl
2,670290,2017,poolians.com
3,767400,2017,彼岸领域
5,772540,2018,trickjump games ltd


In [55]:
df_best_developer_year_review.drop("user_id",axis=1,inplace=True)

In [56]:
df_best_developer_year_review.head(5)

,item_id,recommend,sentiment_analysis
0,1250,True,2
1,22200,True,2
2,43110,True,2
3,251610,True,2
4,227300,True,2


* hacemos la union de la tabla de games y reviews

In [57]:
df_best_developer_year_final = pd.merge(df_best_developer_year_games,df_best_developer_year_review,on="item_id",sort=True)

* reseteamos el index y borramos el item_id

In [58]:
df_best_developer_year_final.reset_index(drop=True,inplace=True)

* hacemos una vista final de como quedó la tabla

In [59]:
df_best_developer_year_final.head()

,item_id,año,developer,recommend,sentiment_analysis
0,10,2000,valve,True,2
1,10,2000,valve,True,1
2,10,2000,valve,True,2
3,10,2000,valve,True,2
4,10,2000,valve,True,2


### consulta 5: def developer_reviews_analysis (developer: str), se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

* borramos las filas con sentiment_analyst neutrales que es 1 porque no lo usamos en la consulta 4 ni 5

In [60]:
df_best_developer_year_final = df_best_developer_year_final[df_best_developer_year_final["sentiment_analysis"] != 1]
df_best_developer_year_final

,item_id,año,developer,recommend,sentiment_analysis
0,10,2000,valve,True,2
2,10,2000,valve,True,2
3,10,2000,valve,True,2
4,10,2000,valve,True,2
6,10,2000,valve,True,2
...,...,...,...,...,...
49415,521340,2016,vladimir maslov,True,2
49416,521430,2016,david mulder,True,2
49417,521570,2016,tamationgames,True,0
49418,521570,2016,tamationgames,True,0


# Exportamos los Endpoints

* endpoint de la consulta 1

In [61]:
df_developer.to_parquet("../Datasets_endpoints/endpoint_Developer.parquet")

* endpoint de la consulta 2

In [62]:
df_userdata_final.to_parquet("../Datasets_endpoints/endpoint_userdata.parquet")

* endpoint de la consulta 3

In [63]:
df_userforgenre_final.to_parquet("../Datasets_endpoints/enpoint_userforgenre.parquet")

* endpoint de la consulta 4 y 5

In [64]:
df_best_developer_year_final.to_parquet("../Datasets_endpoints/endpoint_games_reviews.parquet")